In [ ]:
import os
import sys

from dotenv import find_dotenv, load_dotenv

if os.path.dirname(find_dotenv()) not in sys.path:
    sys.path.append(os.path.dirname(find_dotenv()))

from src.utils import llm as llm_util
from src.utils import debug as debug_util

load_dotenv(override=True)
VERBOSE = debug_util.string_to_bool(os.environ.get("VERBOSE"))

llm = llm_util.create_llm("openai", "gpt-3.5-turbo", verbose=VERBOSE, temperature=0)

## Tools

In [ ]:
import uuid

import boto3
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_community.tools import DuckDuckGoSearchRun


bedrock_agent_runtime_client = boto3.client(
    service_name="bedrock-agent-runtime", region_name=os.environ.get("AWS_DEFAULT_REGION")
)
agent_id = "FPS14ZUTG8"
session_id = uuid.uuid4().hex


@tool
def query_aws(question: str) -> str:
    """Useful to answer questions about EC2 instance.

    :param question: An EC2 relevant question.
    :return: The answer to the queston.
    """
    response = bedrock_agent_runtime_client.invoke_agent(
        agentId=agent_id,
        agentAliasId="TSTALIASID",
        sessionId=session_id,
        inputText=question,
    )

    result = ""

    for event in response.get("completion"):
        chunk = event["chunk"]
        result += chunk["bytes"].decode()

    return result


tools = [DuckDuckGoSearchRun(), query_aws]

## Agent

In [ ]:
from langgraph.prebuilt import create_react_agent

react_agent = create_react_agent(llm, tools)

In [ ]:
question = "How can I create an EC2 instance?"
# question = "Find information about AWS Community Day New Zealand 2024"

In [ ]:
res = react_agent.invoke({"messages": [HumanMessage(content=question)]})

In [ ]:
res["messages"][-1].pretty_print()

In [ ]:
for chunk in react_agent.stream({"messages": [HumanMessage(content=question)]}):
    for key, value in chunk.items():
        print(f"Output from node '{key}':")
        value["messages"][-1].pretty_print()
        print("\n---\n")

## Tracing

LangSmith: <https://smith.langchain.com/o/9efc60a1-b662-5c22-ad65-9b96d88de54e/>